In [ ]:
sc

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession \
    .builder \
    .appName("readCsv") \
    .getOrCreate()

In [ ]:
df = spark.read.option("header","true").csv('s3://sparkbucket123/1500000 Sales Records.csv')
customSchema = df.schema
cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
df_agg = df.filter(F.col("Region") == "Europe" ).groupby("Country").agg(cnt_cond(F.col('Sales Channel') == 'Offline' ).alias('offline_transactions'), cnt_cond(F.col('Sales Channel') == 'Online' ).alias('online_transactions'))
df_sort = df_agg.sort(F.col("offline_transactions").desc())
df_sort.show(3)